<a href="https://colab.research.google.com/github/RamzNN/RamzNN/blob/main/filter_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas openpyxl

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Работа/Платформа-парсер "Сапфир"/stage_one/выгрузка_вк.xlsx')

In [3]:
df.tail()

,group_id,name,description,Subscribers Count,relevance,Threat assesment status,city,country,tags,subcultures
54278,56230843,?? ФК Спартак Москва ?? Красно-белый НАВСЕГДА!,NaN,4699,True,destructive,Москва,Россия,ФК Спартак,Группы футбольных хулиганов
54279,62454446,Ролевая фурри,NaN,6024,True,destructive,NaN,Россия,Фурри,Антисемейные ценности
54280,60343205,НОВИЙ ПОРЯДОК,NaN,11818,True,destructive,Киев,Украина,Український порядок,Политический экстремизм
54281,56146201,Великі Вуйки/Останній москаль,NaN,23295,True,destructive,Киев,Украина,Москаль,Политический экстремизм
54282,55284725,Команда Навального,NaN,205372,True,destructive,NaN,NaN,Навальный,Политический экстремизм


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54283 entries, 0 to 54282
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   group_id                 53583 non-null  object
 1   name                     54280 non-null  object
 2   description              40436 non-null  object
 3   Subscribers Count        53887 non-null  object
 4   relevance                54283 non-null  bool  
 5   Threat assesment status  53968 non-null  object
 6   city                     51682 non-null  object
 7   country                  52236 non-null  object
 8   tags                     54283 non-null  object
 9   subcultures              54283 non-null  object
dtypes: bool(1), object(9)
memory usage: 3.8+ MB


In [11]:
# df.drop_duplicates(subset=['name', 'description', 'relevance'], inplace=True)
# df.info()

In [5]:
# Проверка пропусков
print("\nКоличество пропусков по столбцам:")
print(df.isnull().sum())

# Распределение целевой переменной 'relevance'
print("\nРаспределение 'relevance':")
print(df['relevance'].value_counts())


Количество пропусков по столбцам:
group_id                     700
name                           3
description                13847
Subscribers Count            396
relevance                      0
Threat assesment status      315
city                        2601
country                     2047
tags                           0
subcultures                    0
dtype: int64

Распределение 'relevance':
relevance
False    45142
True      9141
Name: count, dtype: int64


In [6]:
# Если name пустое — заполним пустой строкой
df['name'] = df['name'].fillna('')
# Если description пустое — заполним пустой строкой
df['description'] = df['description'].fillna('')

In [7]:
# Проверка пропусков
print("\nКоличество пропусков по столбцам:")
print(df.isnull().sum())

# Распределение целевой переменной 'relevance'
print("\nРаспределение 'relevance':")
print(df['relevance'].value_counts())


Количество пропусков по столбцам:
group_id                    700
name                          0
description                   0
Subscribers Count           396
relevance                     0
Threat assesment status     315
city                       2601
country                    2047
tags                          0
subcultures                   0
dtype: int64

Распределение 'relevance':
relevance
False    45142
True      9141
Name: count, dtype: int64


In [8]:
# Объединение названия и описания
df['text'] = df['name'].astype(str) + ' ' + df['description'].astype(str)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54283 entries, 0 to 54282
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   group_id                 53583 non-null  object
 1   name                     54283 non-null  object
 2   description              54283 non-null  object
 3   Subscribers Count        53887 non-null  object
 4   relevance                54283 non-null  bool  
 5   Threat assesment status  53968 non-null  object
 6   city                     51682 non-null  object
 7   country                  52236 non-null  object
 8   tags                     54283 non-null  object
 9   subcultures              54283 non-null  object
 10  text                     54283 non-null  object
dtypes: bool(1), object(10)
memory usage: 4.2+ MB


### Обработка текста

В этот раз я попробую без очистки и обработки текста но потом надо будет это сделать и сравнить качество

```Python
def clean_text(text):
    # Приводим к нижнему регистру
    text = text.lower()
    # Удаляем нежелательные символы (можно кастомизировать)
    text = re.sub(r'[^а-яa-z0-9ё\s.,!?–-]', ' ', text)
    # Убираем лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['text'] = df['text'].apply(clean_text)

```

In [10]:
print(df['relevance'].value_counts())

relevance
False    45142
True      9141
Name: count, dtype: int64


In [11]:
df['text_length'] = df['text'].apply(lambda x: len(x.split()))
df['text_length'].describe()

,text_length
count,54283.000000
mean,65.324061
std,112.336866
min,1.000000
25%,4.000000
50%,18.000000
75%,71.000000
max,734.000000


In [12]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['relevance'])
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42, stratify=train_df['relevance'])

### Установка нужных библиотек

In [13]:
!pip install --upgrade pip
!pip install transformers accelerate evaluate torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 171.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 176.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 M

In [14]:
import pandas as pd
import numpy as np
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

# Убедимся, что у нас есть CUDA (если доступно)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Предположим, вы уже прочитали df, train_df, val_df, test_df
# как мы делали ранее

Using device: cuda


In [15]:
train_df['relevance'] = train_df['relevance'].astype(int)
val_df['relevance'] = val_df['relevance'].astype(int)
test_df['relevance'] = test_df['relevance'].astype(int)

In [16]:
model_name = "ai-forever/sbert_large_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2  # бинарная классификация
)
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/sbert_large_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((

### Подготовка данных для Trainer

In [31]:
MAX_LEN = 256  # Так как средняя длина ~65 слов, но max ~734, выберем 512

def tokenize_function(examples):
    # examples – это dict с ключами "text" и "relevance"
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding="max_length",  # или "longest", но max_length=512 безопаснее для batched training
    )


In [32]:
# Для Trainer нужно, чтобы в словаре для каждой строчки были хотя бы поля: 'text' и 'labels' (или 'label').

train_data = {
    "text": train_df["text"].tolist(),
    "labels": train_df["relevance"].tolist()
}

val_data = {
    "text": val_df["text"].tolist(),
    "labels": val_df["relevance"].tolist()
}

test_data = {
    "text": test_df["text"].tolist(),
    "labels": test_df["relevance"].tolist()
}


In [33]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)
test_dataset = Dataset.from_dict(test_data)

# Применяем токенизацию
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Фильтруем ненужные колонки и переименовываем labels -> label (или наоборот)
train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])
# Теперь в каждом датасете есть "input_ids", "attention_mask", (возможно "token_type_ids") и "labels".


Map:   0%|          | 0/43968 [00:00<?, ? examples/s]

Map:   0%|          | 0/4886 [00:00<?, ? examples/s]

Map:   0%|          | 0/5429 [00:00<?, ? examples/s]

### Настройка Trainer

In [34]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="binary")

    return {
        "accuracy": acc["accuracy"],
        "f1": f1["f1"]
    }

In [35]:
batch_size = 8  # Можно увеличить, если хватает GPU (16/32).
num_epochs = 3  # Чаще всего 3-5 эпох хватает для fine-tuning
learning_rate = 1e-5

training_args = TrainingArguments(
    output_dir="my_sbert_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Следим за f1
    greater_is_better=True,
    save_total_limit=2,
    logging_steps=100,               # Логировать каждые 100 шагов
    logging_dir="./logs",            # Директория для логов (если нужна)
    report_to="none",                # Чтобы логи шли просто в консоль
    disable_tqdm=False               # Разрешить tqdm-прогресс‐бар
    # FP16 ?
    # fp16=True,  # Если GPU поддерживает half precision, можно ускорить
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [36]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,  # Чтобы Trainer мог делать pad/collate
    compute_metrics=compute_metrics
)

<ipython-input-36-0825ef664909>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [37]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 